<a href="https://colab.research.google.com/github/yendruamarendra/Delta/blob/main/delta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 200 kB 51.5 MB/s 
     |████████████████████████████████| 199 kB 15.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=6f6419e3c479d5d8e34ec15a64593f785399f49fea8bff08931dc05e985cb792
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.shell import spark
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from datetime import datetime

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.1
      /_/

Using Python version 3.8.16 (default, Dec  7 2022 01:12:13)
Spark context Web UI available at http://3a1c610c4b7a:4040
Spark context available as 'sc' (master = local[*], app id = local-1671553063599).
SparkSession available as 'spark'.


In [5]:
spark =  SparkSession.builder.appName("Delta_flights").getOrCreate()

In [7]:
df = spark.read.csv("/content/Delta/Dummy_Flight_Leg_Data.csv",header = True, inferSchema=True)

In [89]:
df.show()

+------------+---------+---------+---------+---------+------------+--------+----------------+
|Carrier Code|flight_dt|flightnum|orig_arpt|dest_arpt|flightstatus|lastupdt|       flightkey|
+------------+---------+---------+---------+---------+------------+--------+----------------+
|          DL| 01/01/19|       15|      ATL|      TPA|    Boarding|   19:48| DL4346615ATLTPA|
|          DL| 01/01/19|       31|      ATL|      DFW|    Boarding|   19:17| DL4346631ATLDFW|
|          DL| 01/01/19|       91|      ATL|      RSW|    Boarding|   17:20| DL4346691ATLRSW|
|          DL| 01/01/19|      131|      ATL|      GSP|    Boarding|   17:48|DL43466131ATLGSP|
|          DL| 01/01/19|      186|      ATL|      MCO|    Boarding|   15:54|DL43466186ATLMCO|
|          DL| 01/01/19|      214|      ATL|      JFK|    Boarding|   18:22|DL43466214ATLJFK|
|          DL| 01/01/19|      296|      ATL|      RDU|    Boarding|   18:00|DL43466296ATLRDU|
|          DL| 01/01/19|      301|      ATL|      MCI|    Bo

In [8]:
df = df.filter(f.col("flightkey").isNotNull())

In [9]:
windowSpec = Window.partitionBy("flightkey").orderBy(f.col("flight_dt").desc(),f.col("lastupdt").desc())

In [10]:
df_res = df.withColumn("row_number",row_number().over(windowSpec)).filter("row_number=1").drop("row_number")

In [11]:
df_res.show()

+------------+---------+---------+---------+---------+------------+--------+-----------------+
|Carrier Code|flight_dt|flightnum|orig_arpt|dest_arpt|flightstatus|lastupdt|        flightkey|
+------------+---------+---------+---------+---------+------------+--------+-----------------+
|          DL| 01/01/19|     1038|      ATL|      ROC|    Boarding|    9:28|DL434661038ATLROC|
|          DL| 01/01/19|     1048|      ATL|      RSW|    Boarding|    7:34|DL434661048ATLRSW|
|          DL| 01/01/19|     1050|      ATL|      DEN|    Boarding|    9:26|DL434661050ATLDEN|
|          DL| 01/01/19|     1052|      ATL|      MKE|    Boarding|   14:05|DL434661052ATLMKE|
|          DL| 01/01/19|     1053|      ATL|      PBI|    Boarding|   16:04|DL434661053ATLPBI|
|          DL| 01/01/19|     1058|      ATL|      MCO|    Boarding|   11:49|DL434661058ATLMCO|
|          DL| 01/01/19|     1063|      ATL|      DTW|    Boarding|   11:01|DL434661063ATLDTW|
|          DL| 01/01/19|     1065|      ATL|      